In [1]:
# CELL 1 — No pip installs needed! MediaPipe runs in browser via CDN
print('✅ Ready! Run Cell 2 to launch the zero-lag interface.')

✅ Ready! Run Cell 2 to launch the zero-lag interface.


In [2]:
"""
╔══════════════════════════════════════════════════════════════════════════════╗
║  CELL 2 — CV Gesture System Ultra                                           ║
║  Uses google.colab.output + local tunnel so camera works in new window      ║
╚══════════════════════════════════════════════════════════════════════════════╝

HOW IT WORKS:
  • Writes the full HTML app to disk
  • Starts a lightweight Python HTTP server on port 8765
  • Uses colab's built-in port-forwarding (no extra installs) to get an
    https:// URL → camera access is allowed
  • Displays a clickable link + launch button in the cell output
"""

# ── Cell 2: Run this ──────────────────────────────────────────────────────────

import os, threading, http.server, socketserver
from IPython.display import display, HTML
from google.colab.output import eval_js

PORT = 8765

# ═══════════════════════════════════════════════════════════════════════════════
# THE FULL APP HTML  (self-contained, camera-ready)
# ═══════════════════════════════════════════════════════════════════════════════
APP_HTML = r"""<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8">
<title>CV Gesture System Ultra</title>
<style>
@import url('https://fonts.googleapis.com/css2?family=Rajdhani:wght@400;600;700&family=Share+Tech+Mono&display=swap');
:root{
  --bg:#05050f;--bg2:#0a0a1a;--bg3:#0f0f22;
  --border:#1a1a35;--border2:#252545;
  --accent:#00c8ff;--accent2:#7c3aed;--accent3:#00ff88;
  --text:#ccd6f6;--dim:#445577;--danger:#ff4757;--warn:#ffa502;
  --card-glow:0 0 30px rgba(0,200,255,0.06);
}
*,*::before,*::after{box-sizing:border-box;margin:0;padding:0}
body{
  background:var(--bg);
  background-image:radial-gradient(ellipse at 20% 50%,rgba(0,100,255,0.04) 0%,transparent 60%),
                   radial-gradient(ellipse at 80% 20%,rgba(124,58,237,0.04) 0%,transparent 60%);
  font-family:'Rajdhani',sans-serif;color:var(--text);
  min-height:100vh;display:flex;flex-direction:column;
  align-items:center;padding:12px 8px;gap:10px;
}
#header{text-align:center;width:100%}
#header h1{
  font-size:24px;font-weight:700;letter-spacing:2px;text-transform:uppercase;
  background:linear-gradient(90deg,#00c8ff,#7c3aed,#00ff88);
  -webkit-background-clip:text;-webkit-text-fill-color:transparent;
  background-clip:text;background-size:200% auto;animation:gradShift 4s linear infinite;
}
#header p{font-family:'Share Tech Mono',monospace;font-size:11px;color:var(--dim);letter-spacing:1.5px;margin-top:2px}
@keyframes gradShift{0%{background-position:0% center}100%{background-position:200% center}}
#main-layout{display:flex;gap:12px;width:100%;max-width:1240px;align-items:flex-start}
#cam-col{flex:1;display:flex;flex-direction:column;gap:8px;min-width:0}
#canvas-wrapper{
  position:relative;width:100%;border-radius:16px;overflow:hidden;
  border:1.5px solid var(--border2);background:#000;
  box-shadow:var(--card-glow),0 0 0 1px var(--border);
}
#canvas-wrapper::before{
  content:'';position:absolute;inset:0;border-radius:16px;
  border:1px solid rgba(0,200,255,0.1);pointer-events:none;z-index:20;
}
#videoEl{display:none}
#camCanvas{display:block;width:100%;min-height:460px;background:#000}
#drawCanvas,#shapeCanvas,#uiCanvas{position:absolute;top:0;left:0;width:100%;height:100%;pointer-events:none}
#shapeCanvas{z-index:5}
#uiCanvas{z-index:15}
.hud{
  position:absolute;background:rgba(5,5,15,0.82);backdrop-filter:blur(10px);
  border:1px solid var(--border2);border-radius:9px;padding:5px 12px;
  font-family:'Share Tech Mono',monospace;font-size:12px;z-index:18;
}
#gesture-overlay{top:10px;left:10px;display:flex;align-items:center;gap:8px;font-size:14px;font-weight:700;border-color:var(--accent);box-shadow:0 0 15px rgba(0,200,255,0.15)}
.dot{width:8px;height:8px;border-radius:50%;background:#333;transition:background .2s;flex-shrink:0}
.dot.active{background:var(--accent3);box-shadow:0 0 8px var(--accent3)}
#fps-badge{top:10px;right:10px;color:var(--accent3)}
#hand-badge{top:10px;right:80px;color:var(--accent)}
#mode-badge{bottom:10px;left:10px;font-size:12px;font-weight:700;letter-spacing:.5px;transition:all .2s}
#mode-badge.draw{border-color:var(--accent3);color:var(--accent3)}
#mode-badge.erase{border-color:var(--danger);color:var(--danger)}
#mode-badge.stamp{border-color:var(--warn);color:var(--warn)}
#mode-badge.shape{border-color:var(--accent2);color:#c084fc}
#shape-badge{bottom:10px;right:10px;color:#c084fc}
#tool-tabs{display:flex;gap:4px;flex-wrap:wrap}
.tab{flex:1;min-width:70px;padding:7px 6px;border:1.5px solid var(--border2);border-radius:8px;background:var(--bg2);color:var(--dim);font-family:'Rajdhani',sans-serif;font-size:12px;font-weight:700;cursor:pointer;text-align:center;transition:all .15s;letter-spacing:.3px}
.tab:hover{color:var(--text);background:var(--bg3)}
.tab.active{border-color:var(--accent);color:var(--accent);background:rgba(0,200,255,0.06)}
#btn-row{display:flex;gap:6px;flex-wrap:wrap}
.btn{flex:1;min-width:60px;padding:8px 6px;border:none;border-radius:8px;cursor:pointer;font-size:12px;font-weight:700;font-family:'Rajdhani',sans-serif;transition:all .15s;letter-spacing:.3px}
#startBtn{background:linear-gradient(135deg,#0ea5e9,#0369a1);color:#fff}
#startBtn:hover{filter:brightness(1.15);transform:translateY(-1px)}
#clearBtn{background:var(--bg3);color:var(--dim);border:1.5px solid var(--border2)}
#clearBtn:hover{color:var(--danger);border-color:var(--danger)}
#undoBtn,#redoBtn{background:var(--bg3);color:var(--dim);border:1.5px solid var(--border2)}
#undoBtn:hover{color:var(--warn);border-color:var(--warn)}
#redoBtn:hover{color:var(--accent3);border-color:var(--accent3)}
#saveBtn{background:linear-gradient(135deg,#059669,#047857);color:#fff}
#saveBtn:hover{filter:brightness(1.15);transform:translateY(-1px)}
#stopBtn{background:linear-gradient(135deg,var(--danger),#be123c);color:#fff;display:none}
#addGestBtn{background:linear-gradient(135deg,var(--accent2),#4c1d95);color:#fff}
#addGestBtn:hover{filter:brightness(1.15);transform:translateY(-1px)}
#lockShapeBtn{background:linear-gradient(135deg,#d97706,#b45309);color:#fff;display:none}
#color-row{display:flex;align-items:center;gap:6px;flex-wrap:wrap}
#color-row label{font-size:11px;color:var(--dim);font-family:'Share Tech Mono',monospace}
.swatch{width:22px;height:22px;border-radius:50%;border:2px solid transparent;cursor:pointer;transition:all .15s}
.swatch:hover{transform:scale(1.25)}
.swatch.sel{border-color:#fff;box-shadow:0 0 8px rgba(255,255,255,.45)}
#customColor{width:24px;height:22px;border:2px solid var(--border2);border-radius:5px;cursor:pointer;background:none;padding:0}
#brushSlider{width:70px;accent-color:var(--accent)}
#brushVal{font-family:'Share Tech Mono',monospace;font-size:10px;color:var(--dim)}
#tool-mode-row{display:flex;gap:6px;align-items:center;flex-wrap:wrap}
.mode-pill{padding:4px 10px;border-radius:20px;border:1.5px solid var(--border2);background:var(--bg2);font-size:11px;font-weight:700;cursor:pointer;transition:all .15s;font-family:'Rajdhani',sans-serif;color:var(--dim)}
.mode-pill.active{border-color:var(--accent);color:var(--accent);background:rgba(0,200,255,0.07)}
#eraserSize{width:60px;accent-color:var(--danger)}
#stroke-info{display:flex;justify-content:space-between;font-family:'Share Tech Mono',monospace;font-size:10px;color:var(--dim)}
#stroke-info span{color:var(--accent);font-weight:700}
#status-bar{font-family:'Share Tech Mono',monospace;font-size:11px;color:var(--accent);background:var(--bg2);border:1px solid var(--border);border-radius:7px;padding:6px 12px;text-align:center;transition:color .3s}
#status-bar.error{color:var(--danger);border-color:#3a0808}
#status-bar.ok{color:var(--accent3);border-color:#043a14}
#side-panel{width:270px;flex-shrink:0;display:flex;flex-direction:column;gap:9px;max-height:calc(100vh - 70px);overflow-y:auto;scrollbar-width:thin;scrollbar-color:var(--border2) transparent}
#side-panel::-webkit-scrollbar{width:3px}
#side-panel::-webkit-scrollbar-thumb{background:var(--border2);border-radius:3px}
.card{background:var(--bg2);border:1px solid var(--border);border-radius:13px;padding:13px 14px;box-shadow:var(--card-glow);flex-shrink:0}
.card h3{font-size:10px;text-transform:uppercase;letter-spacing:1.5px;color:var(--dim);margin-bottom:10px;display:flex;align-items:center;justify-content:space-between}
.badge{font-size:10px;background:var(--bg3);border:1px solid var(--border2);border-radius:5px;padding:2px 7px;color:var(--accent);text-transform:none;letter-spacing:0;font-weight:700}
#live-card{text-align:center;background:linear-gradient(135deg,var(--bg2),var(--bg3));border-color:var(--border2);position:relative;overflow:hidden}
#live-card::before{content:'';position:absolute;top:-50%;left:-50%;width:200%;height:200%;background:conic-gradient(transparent 0deg,rgba(0,200,255,0.04) 90deg,transparent 180deg);animation:spin 8s linear infinite;pointer-events:none}
@keyframes spin{to{transform:rotate(360deg)}}
#live-emoji{font-size:52px;line-height:1.2;transition:all .15s;position:relative}
#live-name{font-size:16px;font-weight:700;color:var(--accent);margin-top:4px}
#live-desc{font-size:11px;color:var(--dim);margin-top:2px}
#conf-label{font-size:10px;color:var(--dim);margin:8px 0 3px;font-family:'Share Tech Mono',monospace}
#conf-bg{background:var(--bg3);border-radius:4px;height:5px;overflow:hidden}
#conf-bar{height:100%;background:linear-gradient(90deg,var(--accent),var(--accent3));border-radius:4px;transition:width .2s;width:0%}
#hand-info{margin-top:6px;min-height:22px}
.hand-tag{display:inline-block;background:var(--bg3);border:1px solid var(--border2);border-radius:5px;padding:2px 8px;margin:2px;color:var(--accent);font-size:10px;font-family:'Share Tech Mono',monospace}
.g-row{display:flex;align-items:center;gap:9px;padding:7px 5px;border-bottom:1px solid var(--border);border-radius:7px;transition:background .15s;cursor:default}
.g-row:last-child{border-bottom:none}
.g-row:hover{background:var(--bg3)}
.g-row.active-gesture{background:rgba(0,200,255,0.06);border-left:3px solid var(--accent);padding-left:7px}
.g-row.custom-row.active-gesture{border-left-color:var(--accent2)}
.g-ico{font-size:20px;width:28px;text-align:center;flex-shrink:0}
.g-info{flex:1;min-width:0}
.g-info .gname{font-size:12px;color:var(--text);font-weight:700}
.g-info .gact{font-size:10px;color:var(--dim);margin-top:1px}
.g-del{background:none;border:none;cursor:pointer;font-size:13px;color:var(--border2);padding:2px 5px;border-radius:4px;transition:all .15s;flex-shrink:0}
.g-del:hover{color:var(--danger);background:rgba(255,71,87,0.1)}
.custom-badge{font-size:9px;background:rgba(124,58,237,0.15);border:1px solid rgba(124,58,237,0.3);border-radius:3px;padding:1px 5px;color:#a78bfa}
.shape-btn{display:flex;align-items:center;gap:9px;padding:9px 10px;background:var(--bg3);border:1.5px solid var(--border2);border-radius:9px;cursor:pointer;font-size:13px;color:#aab;font-weight:700;transition:all .15s;margin-bottom:6px;width:100%;text-align:left;font-family:'Rajdhani',sans-serif}
.shape-btn:hover{background:var(--bg2)}
.shape-btn.active{background:rgba(124,58,237,0.1);border-color:var(--accent2);color:#c084fc}
.shape-sub{font-size:10px;color:var(--dim);margin-top:1px}
#shape-opts{margin-top:8px;display:flex;flex-direction:column;gap:6px}
#shape-opts label{font-size:11px;color:var(--dim);display:flex;align-items:center;gap:6px}
#shape-opts input[type=color]{width:26px;height:20px;border:none;border-radius:4px;cursor:pointer}
#shape-opts input[type=range]{flex:1;accent-color:var(--accent2)}
.stamp-btn{display:inline-flex;align-items:center;justify-content:center;width:36px;height:36px;font-size:20px;background:var(--bg3);border:1.5px solid var(--border2);border-radius:8px;cursor:pointer;transition:all .15s}
.stamp-btn:hover{border-color:var(--accent);transform:scale(1.15)}
.stamp-btn.active{border-color:var(--warn);background:rgba(255,165,2,0.1)}
#stamp-grid{display:flex;flex-wrap:wrap;gap:5px}
#pinch-meter{display:flex;align-items:center;gap:8px;font-size:11px;color:var(--dim);font-family:'Share Tech Mono',monospace}
#pinch-bar-bg{flex:1;background:var(--bg3);border-radius:3px;height:5px;overflow:hidden}
#pinch-bar{height:100%;background:linear-gradient(90deg,var(--accent2),var(--danger));width:0%;transition:width .1s}
.tip{font-size:11px;color:#889;line-height:1.65;padding:6px 10px;background:var(--bg3);border-left:3px solid var(--accent);border-radius:0 7px 7px 0;margin-bottom:6px}
.tip b{color:var(--accent)}
.tip:last-child{margin-bottom:0}
#preview-modal{display:none;position:fixed;inset:0;background:rgba(0,0,0,.9);z-index:9999;align-items:center;justify-content:center}
#preview-modal.open{display:flex}
#preview-box{background:var(--bg2);border:2px solid var(--accent);border-radius:18px;padding:22px;display:flex;flex-direction:column;align-items:center;gap:14px;max-width:94vw;box-shadow:0 0 60px rgba(0,200,255,0.2)}
#preview-box h2{font-size:18px;color:var(--accent);letter-spacing:1px}
#preview-img{border-radius:10px;border:1px solid var(--border);background:#000;max-width:80vw;max-height:55vh}
#preview-stats{font-family:'Share Tech Mono',monospace;font-size:11px;color:var(--dim);display:flex;gap:18px}
#preview-stats span{color:var(--accent);font-weight:700}
.modal-btns{display:flex;gap:10px}
.modal-btns button{padding:9px 22px;border-radius:8px;border:none;cursor:pointer;font-size:13px;font-weight:700;font-family:'Rajdhani',sans-serif;letter-spacing:.5px}
#downloadBtn{background:var(--accent);color:#000}
#closeModalBtn{background:var(--bg3);color:var(--dim);border:1px solid var(--border2)}
@media(max-width:760px){#main-layout{flex-direction:column}#side-panel{width:100%;max-height:none}}
</style>
</head>
<body>
<div id="header">
  <h1>✋ CV GESTURE SYSTEM ULTRA</h1>
  <p>AIR WRITING · SHAPE DESIGNER · STAMPS · CUSTOM GESTURES · MULTI-TOOL</p>
</div>
<div id="main-layout">
<div id="cam-col">
  <div id="canvas-wrapper">
    <video id="videoEl" autoplay playsinline muted></video>
    <canvas id="camCanvas"></canvas>
    <canvas id="drawCanvas"></canvas>
    <canvas id="shapeCanvas"></canvas>
    <canvas id="uiCanvas"></canvas>
    <div class="hud" id="gesture-overlay"><span class="dot" id="dot"></span><span id="gest-text">Waiting...</span></div>
    <div class="hud" id="fps-badge">-- fps</div>
    <div class="hud" id="hand-badge">0 ✋</div>
    <div class="hud" id="mode-badge">IDLE</div>
    <div class="hud" id="shape-badge">No shape</div>
  </div>
  <div id="status-bar">Click ▶ START to begin · Allow camera access when prompted</div>
  <div id="tool-tabs">
    <div class="tab active" onclick="setToolTab('draw')"  id="tab-draw">✍️ Draw</div>
    <div class="tab"        onclick="setToolTab('shape')" id="tab-shape">🔷 Shapes</div>
    <div class="tab"        onclick="setToolTab('stamp')" id="tab-stamp">🎨 Stamps</div>
    <div class="tab"        onclick="setToolTab('erase')" id="tab-erase">⬜ Erase</div>
  </div>
  <div id="btn-row">
    <button class="btn" id="startBtn"     onclick="startCamera()">▶ START</button>
    <button class="btn" id="clearBtn"     onclick="clearCanvas()">🗑 CLEAR</button>
    <button class="btn" id="undoBtn"      onclick="undo()">↩ UNDO</button>
    <button class="btn" id="redoBtn"      onclick="redo()">↪ REDO</button>
    <button class="btn" id="saveBtn"      onclick="openPreview()">💾 SAVE</button>
    <button class="btn" id="addGestBtn"   onclick="captureGesture()">➕ GESTURE</button>
    <button class="btn" id="lockShapeBtn" onclick="lockCurrentShape()">📌 LOCK</button>
    <button class="btn" id="stopBtn"      onclick="stopCamera()">⏹ STOP</button>
  </div>
  <div id="panel-draw">
    <div id="color-row">
      <label>COLOR:</label>
      <div class="swatch sel" style="background:#00ffaa" data-c="#00ffaa" onclick="setColor(this)"></div>
      <div class="swatch"     style="background:#00c8ff" data-c="#00c8ff" onclick="setColor(this)"></div>
      <div class="swatch"     style="background:#ff6b6b" data-c="#ff6b6b" onclick="setColor(this)"></div>
      <div class="swatch"     style="background:#ffd700" data-c="#ffd700" onclick="setColor(this)"></div>
      <div class="swatch"     style="background:#ff69b4" data-c="#ff69b4" onclick="setColor(this)"></div>
      <div class="swatch"     style="background:#a78bfa" data-c="#a78bfa" onclick="setColor(this)"></div>
      <div class="swatch"     style="background:#fb923c" data-c="#fb923c" onclick="setColor(this)"></div>
      <div class="swatch"     style="background:#ffffff" data-c="#ffffff" onclick="setColor(this)"></div>
      <input type="color" id="customColor" value="#00ffaa" oninput="setCustomColor(this)">
      &nbsp;<label>SIZE:</label>
      <input type="range" id="brushSlider" min="2" max="30" value="4" oninput="onBrushChange()">
      <span id="brushVal">4px</span>
    </div>
    <div id="tool-mode-row">
      <label style="font-size:11px;color:var(--dim)">BRUSH:</label>
      <div class="mode-pill active" onclick="setBrushStyle('round')"  id="pill-round">● Round</div>
      <div class="mode-pill"        onclick="setBrushStyle('square')" id="pill-square">■ Square</div>
      <div class="mode-pill"        onclick="setBrushStyle('glow')"   id="pill-glow">✨ Glow</div>
      <div class="mode-pill"        onclick="setBrushStyle('neon')"   id="pill-neon">⚡ Neon</div>
    </div>
    <div id="stroke-info">
      <span>Strokes: <span id="stroke-count">0</span></span>
      <span>Points: <span id="point-count">0</span></span>
      <span>Undo stack: <span id="undo-count">0</span></span>
    </div>
  </div>
  <div id="panel-erase" style="display:none">
    <div style="display:flex;align-items:center;gap:8px;flex-wrap:wrap">
      <label style="font-size:11px;color:var(--dim)">ERASER SIZE:</label>
      <input type="range" id="eraserSize" min="10" max="80" value="30" oninput="onEraserChange()">
      <span id="eraserVal" style="font-family:'Share Tech Mono',monospace;font-size:10px;color:var(--dim)">30px</span>
    </div>
  </div>
  <div id="panel-stamp" style="display:none">
    <div id="stamp-grid">
      <div class="stamp-btn active" data-stamp="⭐" onclick="setStamp(this)">⭐</div>
      <div class="stamp-btn" data-stamp="❤️"  onclick="setStamp(this)">❤️</div>
      <div class="stamp-btn" data-stamp="🔥"  onclick="setStamp(this)">🔥</div>
      <div class="stamp-btn" data-stamp="💎"  onclick="setStamp(this)">💎</div>
      <div class="stamp-btn" data-stamp="🎯"  onclick="setStamp(this)">🎯</div>
      <div class="stamp-btn" data-stamp="⚡"  onclick="setStamp(this)">⚡</div>
      <div class="stamp-btn" data-stamp="🌈"  onclick="setStamp(this)">🌈</div>
      <div class="stamp-btn" data-stamp="🚀"  onclick="setStamp(this)">🚀</div>
      <div class="stamp-btn" data-stamp="💀"  onclick="setStamp(this)">💀</div>
      <div class="stamp-btn" data-stamp="👑"  onclick="setStamp(this)">👑</div>
      <div class="stamp-btn" data-stamp="🌸"  onclick="setStamp(this)">🌸</div>
      <div class="stamp-btn" data-stamp="✅"  onclick="setStamp(this)">✅</div>
    </div>
    <div style="display:flex;align-items:center;gap:8px;margin-top:8px;font-size:11px;color:var(--dim)">
      STAMP SIZE: <input type="range" id="stamp-size" min="20" max="120" value="50">
      <span id="stamp-size-val" style="font-family:'Share Tech Mono',monospace">50px</span>
    </div>
  </div>
</div>

<div id="side-panel">
  <div class="card" id="live-card">
    <h3>LIVE GESTURE <span class="badge" id="conf-badge">0%</span></h3>
    <div id="live-emoji">👋</div>
    <div id="live-name">No hand detected</div>
    <div id="live-desc">Show your hand to the camera</div>
    <div id="conf-label">Confidence: 0%</div>
    <div id="conf-bg"><div id="conf-bar"></div></div>
    <div id="pinch-meter" style="margin-top:6px">
      <span>PINCH</span><div id="pinch-bar-bg"><div id="pinch-bar"></div></div><span id="pinch-val">0%</span>
    </div>
    <div id="hand-info">—</div>
  </div>
  <div class="card">
    <h3>GESTURE GUIDE <span class="badge" id="gest-count">8</span></h3>
    <div id="gesture-list">
      <div class="g-row" id="row-Pointing"><span class="g-ico">☝️</span><div class="g-info"><div class="gname">Index Finger</div><div class="gact">✍ Draw / Stamp / Erase</div></div></div>
      <div class="g-row" id="row-Fist"><span class="g-ico">✊</span><div class="g-info"><div class="gname">Fist</div><div class="gact">⏸ Lift pen</div></div></div>
      <div class="g-row" id="row-ThumbsUp"><span class="g-ico">👍</span><div class="g-info"><div class="gname">Thumbs Up</div><div class="gact">🗑 Clear canvas</div></div></div>
      <div class="g-row" id="row-OK"><span class="g-ico">👌</span><div class="g-info"><div class="gname">OK Sign</div><div class="gact">⭕ Stamp circle</div></div></div>
      <div class="g-row" id="row-Peace"><span class="g-ico">✌️</span><div class="g-info"><div class="gname">Peace</div><div class="gact">🔷 Shape mode / Lock</div></div></div>
      <div class="g-row" id="row-OpenHand"><span class="g-ico">🖐</span><div class="g-info"><div class="gname">Open Hand</div><div class="gact">⏸ Pause</div></div></div>
      <div class="g-row" id="row-RockOn"><span class="g-ico">🤘</span><div class="g-info"><div class="gname">Rock On</div><div class="gact">↩ Undo</div></div></div>
      <div class="g-row" id="row-ThumbsDown"><span class="g-ico">👎</span><div class="g-info"><div class="gname">Thumbs Down</div><div class="gact">🎨 Cycle color</div></div></div>
    </div>
  </div>
  <div class="card" id="shape-tab-card" style="display:none">
    <h3>SHAPE DESIGNER</h3>
    <button class="shape-btn" id="sb-circle"    onclick="selectShape('circle')"><span>⭕</span><div><div>Circle</div><div class="shape-sub">2 hands = resize · OK = stamp</div></div></button>
    <button class="shape-btn" id="sb-rectangle" onclick="selectShape('rectangle')"><span>▭</span><div><div>Rectangle</div><div class="shape-sub">2 hands = corner + size</div></div></button>
    <button class="shape-btn" id="sb-triangle"  onclick="selectShape('triangle')"><span>🔺</span><div><div>Triangle</div><div class="shape-sub">2 hands = apex + base</div></div></button>
    <button class="shape-btn" id="sb-star"      onclick="selectShape('star')"><span>⭐</span><div><div>Star</div><div class="shape-sub">2 hands = center + size</div></div></button>
    <button class="shape-btn" id="sb-arrow"     onclick="selectShape('arrow')"><span>➡️</span><div><div>Arrow</div><div class="shape-sub">2 hands = start + end</div></div></button>
    <button class="shape-btn" onclick="selectShape(null)" style="opacity:.55;font-size:12px">✕ Deselect</button>
    <div id="shape-opts">
      <label>Fill: <input type="color" id="shapeFill" value="#7c3aed"></label>
      <label>Stroke: <input type="color" id="shapeStroke" value="#c084fc"></label>
      <label>Opacity:
        <input type="range" id="shapeAlpha" min="0" max="100" value="30" oninput="document.getElementById('shapeAlphaLbl').textContent=this.value+'%'">
        <span id="shapeAlphaLbl" style="font-family:'Share Tech Mono',monospace;font-size:10px">30%</span>
      </label>
    </div>
  </div>
  <div class="card">
    <h3>LOCKED SHAPES <span class="badge" id="locked-count">0</span></h3>
    <div id="locked-list" style="font-size:11px;color:var(--dim)">No shapes locked yet.</div>
  </div>
  <div class="card">
    <h3>QUICK TIPS</h3>
    <div class="tip"><b>☝️ Pointing</b> — Draws, stamps, or erases by active mode tab</div>
    <div class="tip"><b>👌 OK</b> — Stamps a permanent glowing circle at that spot</div>
    <div class="tip"><b>✌️ Peace + 2 hands</b> — Locks shape onto canvas</div>
    <div class="tip"><b>🤘 Rock On</b> — Undo · <b>👎 Down</b> — Cycle color</div>
    <div class="tip"><b>➕ GESTURE</b> — Hold pose → click → name → detected live</div>
    <div class="tip"><b>Keyboard:</b> Ctrl+Z undo · Space clear · 1-4 switch mode</div>
  </div>
</div>
</div>

<div id="preview-modal">
  <div id="preview-box">
    <h2>💾 CANVAS EXPORT</h2>
    <canvas id="preview-img"></canvas>
    <div id="preview-stats">Strokes: <span id="modal-strokes">0</span> &nbsp; Points: <span id="modal-points">0</span> &nbsp; Size: <span id="modal-size">—</span></div>
    <div class="modal-btns">
      <button id="downloadBtn"   onclick="downloadDrawing()">⬇️ Download PNG</button>
      <button id="closeModalBtn" onclick="closePreview()">✕ Close</button>
    </div>
  </div>
</div>

<script src="https://cdn.jsdelivr.net/npm/@mediapipe/camera_utils/camera_utils.js" crossorigin="anonymous"></script>
<script src="https://cdn.jsdelivr.net/npm/@mediapipe/hands/hands.js" crossorigin="anonymous"></script>
<script>
"use strict";
const COLORS=["#00ffaa","#00c8ff","#ff6b6b","#ffd700","#ff69b4","#a78bfa","#fb923c","#ffffff"];
let colorIdx=0,drawColor=COLORS[0],brushSize=4,brushStyle="round",eraserSize=30,toolMode="draw";
let isDrawing=false,lastPt=null,smoothPts=[],strokeCount=0,pointCount=0;
const SMOOTH_N=5;
let currentStamp="⭐",stampSize=50,stampCooldown=false,stampCooldownMs=350;
let selectedShape=null,lockedShapes=[];
let okCircleCooldown=false;
const MAX_UNDO=20;let undoStack=[],redoStack=[];
let customGestures=loadCustomGestures();
let mpHands=null,stream=null,rafId=null,latestLandmarks=[];
let fpsFrames=0,fpsLast=performance.now();

const videoEl=document.getElementById("videoEl");
const camCvs=document.getElementById("camCanvas"),drawCvs=document.getElementById("drawCanvas");
const shapeCvs=document.getElementById("shapeCanvas"),uiCvs=document.getElementById("uiCanvas");
const camCtx=camCvs.getContext("2d"),drawCtx=drawCvs.getContext("2d");
const shapeCtx=shapeCvs.getContext("2d"),uiCtx=uiCvs.getContext("2d");

function resizeCanvases(w,h){[camCvs,drawCvs,shapeCvs,uiCvs].forEach(c=>{c.width=w;c.height=h;});drawCtx.lineCap=drawCtx.lineJoin="round";}

const BUILT_IN={Pointing:{emoji:"☝️",name:"Pointing",desc:"Active tool mode",draw:true},Fist:{emoji:"✊",name:"Fist",desc:"Pen lifted",draw:false},ThumbsUp:{emoji:"👍",name:"Thumbs Up",desc:"Canvas cleared",draw:false},ThumbsDown:{emoji:"👎",name:"Thumbs Down",desc:"Color cycled",draw:false},OpenHand:{emoji:"🖐",name:"Open Hand",desc:"Paused",draw:false},Peace:{emoji:"✌️",name:"Peace",desc:"Shape mode on",draw:false},RockOn:{emoji:"🤘",name:"Rock On",desc:"Undo triggered",draw:false},OK:{emoji:"👌",name:"OK Sign",desc:"Circle stamped",draw:false},Unknown:{emoji:"🤚",name:"Detected",desc:"Unknown pose",draw:false},None:{emoji:"👋",name:"No hand",desc:"",draw:false}};
let GESTURES={...BUILT_IN};
function rebuildGestureTable(){GESTURES={...BUILT_IN};for(const[k,cg]of Object.entries(customGestures))GESTURES[k]={emoji:cg.emoji,name:cg.name,desc:"Custom",draw:cg.draw||false};updateGestCount();}

function classifyGesture(lm){
  const thumb=lm[4].x<lm[3].x,index=lm[8].y<lm[6].y,middle=lm[12].y<lm[10].y,ring=lm[16].y<lm[14].y,pinky=lm[20].y<lm[18].y;
  if(thumb&&!index&&!middle&&!ring&&!pinky)return lm[4].y<lm[0].y?"ThumbsUp":"ThumbsDown";
  if(!thumb&&!index&&!middle&&!ring&&!pinky)return"Fist";
  if(index&&middle&&ring&&pinky)return"OpenHand";
  if(index&&!middle&&!ring&&!pinky)return"Pointing";
  if(index&&middle&&!ring&&!pinky)return"Peace";
  if(index&&!middle&&!ring&&pinky)return"RockOn";
  const ok=Math.hypot(lm[4].x-lm[8].x,lm[4].y-lm[8].y);
  if(ok<0.08&&middle&&ring&&pinky)return"OK";
  return matchCustomGesture(lm);
}
function normalizeLandmarks(lm){const wx=lm[0].x,wy=lm[0].y,sc=Math.hypot(lm[9].x-wx,lm[9].y-wy)||1;return lm.map(p=>({x:(p.x-wx)/sc,y:(p.y-wy)/sc}));}
function matchCustomGesture(lm){let bk="Unknown",bd=0.10;for(const[k,cg]of Object.entries(customGestures)){if(!cg.landmarks||cg.landmarks.length!==21)continue;const n=normalizeLandmarks(lm),r=cg.landmarks;let d=0;for(let i=1;i<21;i++)d+=Math.hypot(n[i].x-r[i].x,n[i].y-r[i].y);d/=20;if(d<bd){bd=d;bk=k;}}return bk;}

const gestureFired={};
function fireOnce(key,fn,ms=600){if(gestureFired[key])return;gestureFired[key]=true;fn();setTimeout(()=>{gestureFired[key]=false;},ms);}

const CONNECTIONS=[[0,1],[1,2],[2,3],[3,4],[0,5],[5,6],[6,7],[7,8],[0,9],[9,10],[10,11],[11,12],[0,13],[13,14],[14,15],[15,16],[0,17],[17,18],[18,19],[19,20],[5,9],[9,13],[13,17]];
function drawSkeleton(lm,W,H,bc){
  camCtx.save();camCtx.strokeStyle=bc;camCtx.lineWidth=2;camCtx.lineCap="round";
  CONNECTIONS.forEach(([a,b])=>{camCtx.beginPath();camCtx.moveTo(lm[a].x*W,lm[a].y*H);camCtx.lineTo(lm[b].x*W,lm[b].y*H);camCtx.stroke();});
  lm.forEach((p,i)=>{const tip=[4,8,12,16,20].includes(i);camCtx.beginPath();camCtx.arc(p.x*W,p.y*H,tip?6:3.5,0,Math.PI*2);camCtx.fillStyle=tip?"#00ffaa":bc;camCtx.fill();camCtx.strokeStyle="rgba(255,255,255,0.6)";camCtx.lineWidth=1;camCtx.stroke();});
  const fx=lm[8].x*W,fy=lm[8].y*H;
  if(toolMode==="draw"&&isDrawing){const g=camCtx.createRadialGradient(fx,fy,0,fx,fy,22);g.addColorStop(0,hexAlpha(drawColor,0.55));g.addColorStop(1,hexAlpha(drawColor,0));camCtx.beginPath();camCtx.arc(fx,fy,22,0,Math.PI*2);camCtx.fillStyle=g;camCtx.fill();}
  if(toolMode==="erase"){camCtx.beginPath();camCtx.arc(fx,fy,eraserSize/2,0,Math.PI*2);camCtx.strokeStyle="rgba(255,70,70,0.8)";camCtx.lineWidth=2;camCtx.setLineDash([5,3]);camCtx.stroke();camCtx.setLineDash([]);}
  if(toolMode==="stamp"){camCtx.font=`${stampSize*0.5}px serif`;camCtx.textAlign="center";camCtx.textBaseline="middle";camCtx.globalAlpha=0.5;camCtx.fillText(currentStamp,fx,fy);camCtx.globalAlpha=1;}
  camCtx.restore();
}

function renderLiveShape(h1,h2,W,H){
  shapeCtx.clearRect(0,0,shapeCvs.width,shapeCvs.height);if(!selectedShape||!h1||!h2)return;
  const ax=h1[0].x*W,ay=h1[0].y*H,bx=h2[0].x*W,by=h2[0].y*H;
  const dist=Math.hypot(bx-ax,by-ay),angle=Math.atan2(by-ay,bx-ax);
  const fill=document.getElementById("shapeFill").value,stroke=document.getElementById("shapeStroke").value;
  const alpha=parseInt(document.getElementById("shapeAlpha").value)/100;
  const fr=parseInt(fill.slice(1,3),16),fg=parseInt(fill.slice(3,5),16),fb=parseInt(fill.slice(5,7),16);
  const cx=(ax+bx)/2,cy=(ay+by)/2;
  shapeCtx.save();shapeCtx.shadowBlur=25;shapeCtx.shadowColor=stroke;shapeCtx.strokeStyle=stroke;shapeCtx.fillStyle=`rgba(${fr},${fg},${fb},${alpha})`;shapeCtx.lineWidth=3;shapeCtx.lineCap=shapeCtx.lineJoin="round";
  if(selectedShape==="circle"){shapeCtx.beginPath();shapeCtx.arc(cx,cy,dist/2,0,Math.PI*2);shapeCtx.fill();shapeCtx.stroke();}
  else if(selectedShape==="rectangle"){shapeCtx.save();shapeCtx.translate(cx,cy);shapeCtx.rotate(angle);shapeCtx.beginPath();shapeCtx.rect(-dist/2,-dist/4,dist,dist/2);shapeCtx.fill();shapeCtx.stroke();shapeCtx.restore();}
  else if(selectedShape==="triangle"){const h=dist*0.866,px=-Math.sin(angle)*h/2,py=Math.cos(angle)*h/2;shapeCtx.beginPath();shapeCtx.moveTo(ax,ay);shapeCtx.lineTo(bx+px,by+py);shapeCtx.lineTo(bx-px,by-py);shapeCtx.closePath();shapeCtx.fill();shapeCtx.stroke();}
  else if(selectedShape==="star"){drawStar(shapeCtx,cx,cy,5,dist/2,dist/4,angle);shapeCtx.fill();shapeCtx.stroke();}
  else if(selectedShape==="arrow"){drawArrow(shapeCtx,ax,ay,bx,by,dist*0.15);shapeCtx.stroke();}
  shapeCtx.restore();
  document.getElementById("shape-badge").textContent=`${selectedShape} · ${Math.round(dist)}px · ${Math.round(angle*180/Math.PI)}°`;
  window._liveShapeDef={type:selectedShape,ax,ay,bx,by,dist,angle,cx,cy,fill,stroke,alpha};
}
function drawStar(ctx,cx,cy,sp,or,ir,rot){ctx.beginPath();for(let i=0;i<sp*2;i++){const r=i%2===0?or:ir,a=rot+(i*Math.PI/sp);i===0?ctx.moveTo(cx+Math.cos(a)*r,cy+Math.sin(a)*r):ctx.lineTo(cx+Math.cos(a)*r,cy+Math.sin(a)*r);}ctx.closePath();}
function drawArrow(ctx,x1,y1,x2,y2,hl){const a=Math.atan2(y2-y1,x2-x1);ctx.beginPath();ctx.moveTo(x1,y1);ctx.lineTo(x2,y2);ctx.lineTo(x2-hl*Math.cos(a-Math.PI/6),y2-hl*Math.sin(a-Math.PI/6));ctx.moveTo(x2,y2);ctx.lineTo(x2-hl*Math.cos(a+Math.PI/6),y2-hl*Math.sin(a+Math.PI/6));}

function lockCurrentShape(){if(!window._liveShapeDef){setStatus("⚠️ No active shape! Use 2 hands in Shape mode first.","error");return;}pushUndo();stampShapeOnto(drawCtx,window._liveShapeDef);lockedShapes.push({...window._liveShapeDef});updateLockedList();setStatus("📌 Shape locked!","ok");}
function stampShapeOnto(ctx,d){
  const fr=parseInt(d.fill.slice(1,3),16),fg=parseInt(d.fill.slice(3,5),16),fb=parseInt(d.fill.slice(5,7),16);
  ctx.save();ctx.shadowBlur=20;ctx.shadowColor=d.stroke;ctx.strokeStyle=d.stroke;ctx.fillStyle=`rgba(${fr},${fg},${fb},${d.alpha})`;ctx.lineWidth=3;ctx.lineCap=ctx.lineJoin="round";
  if(d.type==="circle"){ctx.beginPath();ctx.arc(d.cx,d.cy,d.dist/2,0,Math.PI*2);ctx.fill();ctx.stroke();}
  else if(d.type==="rectangle"){ctx.save();ctx.translate(d.cx,d.cy);ctx.rotate(d.angle);ctx.beginPath();ctx.rect(-d.dist/2,-d.dist/4,d.dist,d.dist/2);ctx.fill();ctx.stroke();ctx.restore();}
  else if(d.type==="triangle"){const h=d.dist*0.866,px=-Math.sin(d.angle)*h/2,py=Math.cos(d.angle)*h/2;ctx.beginPath();ctx.moveTo(d.ax,d.ay);ctx.lineTo(d.bx+px,d.by+py);ctx.lineTo(d.bx-px,d.by-py);ctx.closePath();ctx.fill();ctx.stroke();}
  else if(d.type==="star"){drawStar(ctx,d.cx,d.cy,5,d.dist/2,d.dist/4,d.angle);ctx.fill();ctx.stroke();}
  else if(d.type==="arrow"){drawArrow(ctx,d.ax,d.ay,d.bx,d.by,d.dist*0.15);ctx.stroke();}
  ctx.restore();
}
function updateLockedList(){document.getElementById("locked-count").textContent=lockedShapes.length;const list=document.getElementById("locked-list");if(!lockedShapes.length){list.innerHTML='<span style="color:var(--dim);font-size:11px">No shapes locked yet.</span>';return;}list.innerHTML=lockedShapes.map((s,i)=>`<div style="display:flex;align-items:center;justify-content:space-between;padding:4px 0;border-bottom:1px solid var(--border);font-size:11px"><span style="color:var(--text)">${s.type}</span><button onclick="unlockShape(${i})" style="background:none;border:none;color:var(--dim);cursor:pointer">❌</button></div>`).join('');}
function unlockShape(i){lockedShapes.splice(i,1);updateLockedList();}

function handleOKGesture(lm,W,H){if(okCircleCooldown)return;okCircleCooldown=true;setTimeout(()=>{okCircleCooldown=false;},1200);pushUndo();const mx=((1-lm[4].x)*W+(1-lm[8].x)*W)/2,my=(lm[4].y*H+lm[8].y*H)/2;const r=Math.max(20,Math.hypot((1-lm[4].x)*W-(1-lm[8].x)*W,lm[4].y*H-lm[8].y*H)/2);drawCtx.save();drawCtx.shadowBlur=25;drawCtx.shadowColor=drawColor;const c=hexToRgb(drawColor);drawCtx.fillStyle=`rgba(${c.r},${c.g},${c.b},0.18)`;drawCtx.beginPath();drawCtx.arc(mx,my,r,0,Math.PI*2);drawCtx.fill();drawCtx.strokeStyle=drawColor;drawCtx.lineWidth=4;drawCtx.stroke();drawCtx.restore();strokeCount++;updateStrokeInfo();setStatus(`⭕ Circle stamped r=${Math.round(r)}px`,"ok");}

function getSmoothed(x,y){smoothPts.push({x,y});if(smoothPts.length>SMOOTH_N)smoothPts.shift();return{x:smoothPts.reduce((a,p)=>a+p.x,0)/smoothPts.length,y:smoothPts.reduce((a,p)=>a+p.y,0)/smoothPts.length};}
function drawPoint(x,y,startNew){const sp=getSmoothed(x,y);if(startNew||!lastPt){lastPt=sp;strokeCount++;updateStrokeInfo();return;}
  drawCtx.save();
  if(brushStyle==="round"||brushStyle==="square"){drawCtx.lineCap=brushStyle==="square"?"butt":"round";drawCtx.lineJoin="round";drawCtx.globalAlpha=0.15;drawCtx.strokeStyle=drawColor;drawCtx.lineWidth=brushSize*4;drawCtx.beginPath();drawCtx.moveTo(lastPt.x,lastPt.y);drawCtx.lineTo(sp.x,sp.y);drawCtx.stroke();drawCtx.globalAlpha=1;drawCtx.strokeStyle=drawColor;drawCtx.lineWidth=brushSize;drawCtx.beginPath();drawCtx.moveTo(lastPt.x,lastPt.y);drawCtx.lineTo(sp.x,sp.y);drawCtx.stroke();}
  else if(brushStyle==="glow"){for(let w=brushSize*5;w>=brushSize;w-=brushSize){const c=hexToRgb(drawColor);drawCtx.strokeStyle=`rgba(${c.r},${c.g},${c.b},${0.05+0.1*(brushSize/w)})`;drawCtx.lineWidth=w;drawCtx.lineCap="round";drawCtx.beginPath();drawCtx.moveTo(lastPt.x,lastPt.y);drawCtx.lineTo(sp.x,sp.y);drawCtx.stroke();}}
  else if(brushStyle==="neon"){drawCtx.shadowBlur=20;drawCtx.shadowColor=drawColor;drawCtx.strokeStyle="#fff";drawCtx.lineWidth=brushSize*0.5;drawCtx.lineCap="round";drawCtx.beginPath();drawCtx.moveTo(lastPt.x,lastPt.y);drawCtx.lineTo(sp.x,sp.y);drawCtx.stroke();drawCtx.shadowBlur=0;drawCtx.strokeStyle=drawColor;drawCtx.lineWidth=brushSize;drawCtx.beginPath();drawCtx.moveTo(lastPt.x,lastPt.y);drawCtx.lineTo(sp.x,sp.y);drawCtx.stroke();}
  drawCtx.restore();lastPt=sp;pointCount++;updateStrokeInfo();}
function erasePoint(x,y){drawCtx.clearRect(x-eraserSize/2,y-eraserSize/2,eraserSize,eraserSize);}
function stampAtPoint(x,y){if(stampCooldown)return;stampCooldown=true;setTimeout(()=>{stampCooldown=false;},stampCooldownMs);pushUndo();drawCtx.save();drawCtx.font=`${stampSize}px serif`;drawCtx.textAlign="center";drawCtx.textBaseline="middle";drawCtx.fillText(currentStamp,x,y);drawCtx.restore();strokeCount++;updateStrokeInfo();}

function pushUndo(){if(undoStack.length>=MAX_UNDO)undoStack.shift();undoStack.push(drawCtx.getImageData(0,0,drawCvs.width,drawCvs.height));redoStack=[];document.getElementById("undo-count").textContent=undoStack.length;}
function undo(){if(!undoStack.length){setStatus("⚠️ Nothing to undo","error");return;}redoStack.push(drawCtx.getImageData(0,0,drawCvs.width,drawCvs.height));drawCtx.putImageData(undoStack.pop(),0,0);document.getElementById("undo-count").textContent=undoStack.length;}
function redo(){if(!redoStack.length)return;undoStack.push(drawCtx.getImageData(0,0,drawCvs.width,drawCvs.height));drawCtx.putImageData(redoStack.pop(),0,0);document.getElementById("undo-count").textContent=undoStack.length;}

function onResults(results){
  const W=camCvs.width,H=camCvs.height;
  camCtx.save();camCtx.translate(W,0);camCtx.scale(-1,1);camCtx.drawImage(results.image,0,0,W,H);camCtx.restore();
  camCtx.drawImage(drawCvs,0,0);camCtx.drawImage(shapeCvs,0,0);
  fpsFrames++;const now=performance.now();if(now-fpsLast>=1000){document.getElementById("fps-badge").textContent=fpsFrames+" fps";fpsFrames=0;fpsLast=now;}
  const hands=results.multiHandLandmarks||[];latestLandmarks=hands;
  document.getElementById("hand-badge").textContent=hands.length+" ✋";
  if(!hands.length){updateGestureUI("None",0);isDrawing=false;lastPt=null;smoothPts=[];shapeCtx.clearRect(0,0,shapeCvs.width,shapeCvs.height);document.getElementById("hand-info").textContent="—";return;}
  const lm=hands[0],gesture=classifyGesture(lm);
  const mirrorLm=lm.map(p=>({x:1-p.x,y:p.y,z:p.z}));drawSkeleton(mirrorLm,W,H,"#50b4ff");
  let mirrorLm2=null;if(hands.length>=2){mirrorLm2=hands[1].map(p=>({x:1-p.x,y:p.y,z:p.z}));drawSkeleton(mirrorLm2,W,H,"#ff8050");}
  const pd=Math.hypot(lm[4].x-lm[8].x,lm[4].y-lm[8].y),pp=Math.max(0,Math.min(100,Math.round((1-pd/0.2)*100)));
  document.getElementById("pinch-bar").style.width=pp+"%";document.getElementById("pinch-val").textContent=pp+"%";
  const handedness=results.multiHandedness||[];
  document.getElementById("hand-info").innerHTML=hands.map((_,i)=>`<span class="hand-tag">${handedness[i]?handedness[i].label:"Hand "+(i+1)}</span>`).join("");
  updateGestureUI(gesture,0.92);
  if(selectedShape){renderLiveShape(mirrorLm,mirrorLm2,W,H);if(gesture==="Peace"&&mirrorLm2)fireOnce("lockByGest",()=>lockCurrentShape(),1000);}
  const mirX=v=>(1-v)*W,ix=mirX(lm[8].x),iy=lm[8].y*H;
  switch(gesture){
    case"ThumbsUp":fireOnce("tu",()=>{pushUndo();clearCanvas();},800);break;
    case"ThumbsDown":fireOnce("td",()=>{colorIdx=(colorIdx+1)%COLORS.length;drawColor=COLORS[colorIdx];document.querySelectorAll(".swatch").forEach((s,i)=>s.classList.toggle("sel",i===colorIdx));setStatus("🎨 Color → "+drawColor,"ok");},700);break;
    case"RockOn":fireOnce("ro",()=>undo(),700);break;
    case"OK":handleOKGesture(lm,W,H);break;
    case"Pointing":{const sn=!isDrawing;isDrawing=true;if(toolMode==="draw"){if(sn)pushUndo();drawPoint(ix,iy,sn);}else if(toolMode==="erase")erasePoint(ix,iy);else if(toolMode==="stamp")stampAtPoint(ix,iy);break;}
    default:if(isDrawing){isDrawing=false;lastPt=null;smoothPts=[];}break;
  }
  if(gesture!=="Pointing"&&gesture!=="OK"&&isDrawing){isDrawing=false;lastPt=null;smoothPts=[];}
}

async function startCamera(){
  setStatus("📷 Requesting camera…");
  try{stream=await navigator.mediaDevices.getUserMedia({video:{width:{ideal:1280},height:{ideal:720},frameRate:{ideal:30}}});}
  catch(e){setStatus("❌ Camera denied: "+e.message,"error");return;}
  videoEl.srcObject=stream;await videoEl.play();
  const W=videoEl.videoWidth||1280,H=videoEl.videoHeight||720;resizeCanvases(W,H);
  mpHands=new Hands({locateFile:f=>`https://cdn.jsdelivr.net/npm/@mediapipe/hands/${f}`});
  mpHands.setOptions({maxNumHands:2,modelComplexity:1,minDetectionConfidence:0.65,minTrackingConfidence:0.55});
  mpHands.onResults(onResults);
  setStatus("⚙️ Loading MediaPipe model… (first load ~10s)");await mpHands.initialize();
  setStatus("✅ Ready — show your hand!","ok");
  document.getElementById("startBtn").style.display="none";
  document.getElementById("stopBtn").style.display="inline-block";
  document.getElementById("lockShapeBtn").style.display="inline-block";
  async function detect(){if(!stream)return;await mpHands.send({image:videoEl});rafId=requestAnimationFrame(detect);}
  detect();
}
function stopCamera(){if(rafId){cancelAnimationFrame(rafId);rafId=null;}if(stream){stream.getTracks().forEach(t=>t.stop());stream=null;}if(mpHands){try{mpHands.close();}catch{}mpHands=null;}videoEl.srcObject=null;[camCtx,shapeCtx].forEach(c=>c.clearRect(0,0,camCvs.width,camCvs.height));document.getElementById("startBtn").style.display="inline-block";document.getElementById("stopBtn").style.display="none";setStatus("⏹ Stopped.");}

function loadCustomGestures(){try{const r=localStorage.getItem("cv_gest_v2");return r?JSON.parse(r):{}}catch{return{};}}
function saveCustomGestures(){try{localStorage.setItem("cv_gest_v2",JSON.stringify(customGestures));}catch{}}
function captureGesture(){if(!latestLandmarks||!latestLandmarks.length){alert("⚠️ No hand visible! Show your hand, hold your pose, THEN click Add Gesture.");return;}const name=prompt("Name this gesture:");if(!name||!name.trim())return;const emojis=["🖐","🤙","🖖","🤞","💪","👆","🤏","🫶","🫰","🤚","🫱","🫲","🦾","🙌"];const emoji=emojis[Math.floor(Math.random()*emojis.length)];const key="cg_"+name.trim().replace(/\s+/g,"_").toLowerCase()+"_"+Date.now();customGestures[key]={name:name.trim(),emoji,landmarks:normalizeLandmarks(latestLandmarks[0]).map(p=>({x:p.x,y:p.y})),draw:false};saveCustomGestures();rebuildGestureTable();addCustomGestureRow(key,customGestures[key]);setStatus(`✅ Gesture "${name.trim()}" saved!`,"ok");}
function addCustomGestureRow(key,cg){document.getElementById("row-"+key)?.remove();const row=document.createElement("div");row.className="g-row custom-row";row.id="row-"+key;row.innerHTML=`<span class="g-ico">${cg.emoji}</span><div class="g-info"><div class="gname">${cg.name} <span class="custom-badge">CUSTOM</span></div><div class="gact">🎯 Normalized match</div></div><button class="g-del" onclick="deleteGesture('${key}')">❌</button>`;document.getElementById("gesture-list").appendChild(row);updateGestCount();}
function deleteGesture(key){if(!confirm(`Delete "${customGestures[key]?.name}"?`))return;delete customGestures[key];saveCustomGestures();rebuildGestureTable();document.getElementById("row-"+key)?.remove();updateGestCount();}
function updateGestCount(){document.getElementById("gest-count").textContent=Object.keys(BUILT_IN).length-2+Object.keys(customGestures).length;}

let activeRow=null;
function updateGestureUI(key,conf){const g=GESTURES[key]||GESTURES.Unknown,pct=Math.round((conf||0)*100);document.getElementById("live-emoji").textContent=g.emoji;document.getElementById("live-name").textContent=g.name;document.getElementById("live-desc").textContent=g.desc;document.getElementById("conf-label").textContent="Confidence: "+pct+"%";document.getElementById("conf-bar").style.width=pct+"%";document.getElementById("conf-badge").textContent=pct+"%";document.getElementById("gest-text").textContent=g.emoji+"  "+g.name;document.getElementById("dot").className="dot"+(g.draw?" active":"");if(activeRow)activeRow.classList.remove("active-gesture");const row=document.getElementById("row-"+key);if(row){row.classList.add("active-gesture");activeRow=row;}const mb=document.getElementById("mode-badge");if(g.draw){mb.textContent=toolMode.toUpperCase();mb.className="hud "+toolMode;}else{mb.textContent=key==="None"?"IDLE":key.toUpperCase();mb.className="hud";}}
function setStatus(msg,type=""){const b=document.getElementById("status-bar");b.textContent=msg;b.className=type;}
function setColor(el){document.querySelectorAll(".swatch").forEach(s=>s.classList.remove("sel"));el.classList.add("sel");drawColor=el.dataset.c;}
function setCustomColor(el){drawColor=el.value;document.querySelectorAll(".swatch").forEach(s=>s.classList.remove("sel"));}
function onBrushChange(){brushSize=parseInt(document.getElementById("brushSlider").value);document.getElementById("brushVal").textContent=brushSize+"px";}
function onEraserChange(){eraserSize=parseInt(document.getElementById("eraserSize").value);document.getElementById("eraserVal").textContent=eraserSize+"px";}
function setBrushStyle(s){brushStyle=s;document.querySelectorAll(".mode-pill").forEach(p=>p.classList.remove("active"));document.getElementById("pill-"+s).classList.add("active");}
function setStamp(el){currentStamp=el.dataset.stamp;document.querySelectorAll(".stamp-btn").forEach(b=>b.classList.remove("active"));el.classList.add("active");}
document.getElementById("stamp-size").addEventListener("input",function(){stampSize=parseInt(this.value);document.getElementById("stamp-size-val").textContent=stampSize+"px";});
function setToolTab(tab){toolMode=tab;document.querySelectorAll(".tab").forEach(t=>t.classList.remove("active"));document.getElementById("tab-"+tab).classList.add("active");["draw","erase","stamp"].forEach(p=>{const el=document.getElementById("panel-"+p);if(el)el.style.display=p===tab?"":"none";});document.getElementById("shape-tab-card").style.display=tab==="shape"?"":"none";if(tab!=="shape"){selectedShape=null;shapeCtx.clearRect(0,0,shapeCvs.width,shapeCvs.height);}isDrawing=false;lastPt=null;smoothPts=[];}
function selectShape(s){selectedShape=s;["circle","rectangle","triangle","star","arrow"].forEach(n=>{document.getElementById("sb-"+n)?.classList.toggle("active",n===s);});if(!s){shapeCtx.clearRect(0,0,shapeCvs.width,shapeCvs.height);document.getElementById("shape-badge").textContent="No shape";window._liveShapeDef=null;}}
function clearCanvas(){drawCtx.clearRect(0,0,drawCvs.width,drawCvs.height);strokeCount=0;pointCount=0;updateStrokeInfo();}
function updateStrokeInfo(){document.getElementById("stroke-count").textContent=strokeCount;document.getElementById("point-count").textContent=pointCount;}
function hexToRgb(h){return{r:parseInt(h.slice(1,3),16),g:parseInt(h.slice(3,5),16),b:parseInt(h.slice(5,7),16)};}
function hexAlpha(h,a){const{r,g,b}=hexToRgb(h);return`rgba(${r},${g},${b},${a})`;}
function openPreview(){const p=document.getElementById("preview-img");p.width=drawCvs.width||1280;p.height=drawCvs.height||720;const ctx=p.getContext("2d");ctx.fillStyle="#000";ctx.fillRect(0,0,p.width,p.height);if(camCvs.width>0)ctx.drawImage(camCvs,0,0);ctx.drawImage(drawCvs,0,0);if(shapeCvs.width>0)ctx.drawImage(shapeCvs,0,0);document.getElementById("modal-strokes").textContent=strokeCount;document.getElementById("modal-points").textContent=pointCount;document.getElementById("modal-size").textContent=p.width+"×"+p.height;document.getElementById("preview-modal").classList.add("open");}
function closePreview(){document.getElementById("preview-modal").classList.remove("open");}
function downloadDrawing(){const p=document.getElementById("preview-img"),a=document.createElement("a");a.href=p.toDataURL("image/png");a.download="cv_gesture_"+Date.now()+".png";a.click();}

resizeCanvases(1280,720);rebuildGestureTable();
for(const[k,cg]of Object.entries(customGestures))addCustomGestureRow(k,cg);
document.addEventListener("keydown",e=>{if(e.target.tagName==="INPUT")return;if(e.code==="Space"){e.preventDefault();pushUndo();clearCanvas();}if(e.code==="KeyZ"&&e.ctrlKey){e.preventDefault();undo();}if(e.code==="KeyY"&&e.ctrlKey){e.preventDefault();redo();}if(e.code==="Escape")closePreview();["Digit1","Digit2","Digit3","Digit4"].forEach((c,i)=>{if(e.code===c)setToolTab(["draw","shape","stamp","erase"][i]);});});
</script>
</body>
</html>"""

# ── Write HTML file to disk ───────────────────────────────────────────────────
HTML_PATH = "/tmp/cv_gesture_app.html"
with open(HTML_PATH, "w") as f:
    f.write(APP_HTML)

# ── Start HTTP server in background thread ────────────────────────────────────
class SilentHandler(http.server.SimpleHTTPRequestHandler):
    def log_message(self, *args): pass   # suppress request logs
    def do_GET(self):
        # Always serve the app HTML regardless of path
        self.send_response(200)
        self.send_header("Content-type", "text/html; charset=utf-8")
        # Required headers so camera works in the Colab-proxied window
        self.send_header("Cross-Origin-Opener-Policy", "same-origin")
        self.send_header("Cross-Origin-Embedder-Policy", "require-corp")
        self.end_headers()
        with open(HTML_PATH, "rb") as f:
            self.wfile.write(f.read())

def run_server():
    with socketserver.TCPServer(("", PORT), SilentHandler) as httpd:
        httpd.serve_forever()

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# ── Get the Colab public HTTPS URL for this port ─────────────────────────────
# eval_js tunnels the local port through Colab's proxy → gives an https:// URL
app_url = eval_js(f"google.colab.kernel.proxyPort({PORT})")

# ── Show launcher UI ──────────────────────────────────────────────────────────
display(HTML(f"""
<style>
@import url('https://fonts.googleapis.com/css2?family=Rajdhani:wght@700&family=Share+Tech+Mono&display=swap');
#cv-launcher {{
  background: linear-gradient(135deg, #05050f, #0a0a1a);
  border: 1.5px solid #252545;
  border-radius: 18px;
  padding: 32px 40px;
  text-align: center;
  font-family: 'Rajdhani', sans-serif;
  max-width: 580px;
  margin: 16px auto;
  box-shadow: 0 0 60px rgba(0,200,255,0.08), 0 0 0 1px #1a1a35;
}}
#cv-launcher h1 {{
  font-size: 26px; font-weight: 700; letter-spacing: 3px; text-transform: uppercase;
  background: linear-gradient(90deg, #00c8ff, #7c3aed, #00ff88);
  -webkit-background-clip: text; -webkit-text-fill-color: transparent;
  background-clip: text; margin-bottom: 6px;
}}
#cv-launcher p {{
  font-family: 'Share Tech Mono', monospace;
  font-size: 12px; color: #445577; letter-spacing: 1px; margin-bottom: 24px;
}}
.open-btn {{
  display: inline-block; padding: 16px 52px;
  background: linear-gradient(135deg, #0ea5e9, #7c3aed);
  color: #fff; font-family: 'Rajdhani', sans-serif;
  font-size: 20px; font-weight: 700; letter-spacing: 2px;
  border: none; border-radius: 12px; cursor: pointer;
  text-decoration: none;
  box-shadow: 0 0 30px rgba(14,165,233,0.35);
  transition: all .2s;
}}
.open-btn:hover {{
  transform: translateY(-3px) scale(1.04);
  box-shadow: 0 0 50px rgba(14,165,233,0.55);
}}
.feats {{
  display: flex; flex-wrap: wrap; gap: 7px;
  justify-content: center; margin-top: 20px;
}}
.feat {{
  background: #0f0f22; border: 1px solid #1a1a35;
  border-radius: 20px; padding: 4px 13px;
  font-size: 11px; color: #445577; letter-spacing: .5px;
  font-family: 'Share Tech Mono', monospace;
}}
.url-box {{
  margin-top: 16px; padding: 8px 14px;
  background: #0a0a16; border: 1px solid #1a1a35; border-radius: 8px;
  font-family: 'Share Tech Mono', monospace; font-size: 10px; color: #2a4a8a;
  word-break: break-all;
}}
.note {{
  margin-top: 14px; font-size: 11px; color: #2a2a3a;
  font-family: 'Share Tech Mono', monospace; line-height: 1.7;
}}
</style>

<div id="cv-launcher">
  <h1>✋ CV GESTURE SYSTEM</h1>
  <p>ULTRA · v3.0 · MediaPipe · Real-time · HTTPS ✓</p>

  <a class="open-btn" href="{app_url}" target="_blank" rel="noopener">
    🚀 &nbsp; OPEN FULL SCREEN
  </a>

  <div class="feats">
    <span class="feat">✍️ Air Writing</span>
    <span class="feat">🔷 5 Shapes</span>
    <span class="feat">🎨 12 Stamps</span>
    <span class="feat">➕ Custom Gestures</span>
    <span class="feat">↩ Undo / Redo</span>
    <span class="feat">💾 PNG Export</span>
    <span class="feat">✌️ 2-Hand Mode</span>
    <span class="feat">⚡ Neon Brush</span>
  </div>

  <div class="url-box">🔗 {app_url}</div>

  <div class="note">
    Served via Colab HTTPS proxy → camera access works ✓<br>
    If the tab closes, just re-click the button above — server stays running
  </div>
</div>
"""))

print(f"\n✅ Server running on port {PORT}")
print(f"🔗 App URL: {app_url}")
print("📌 Keep this cell running — the server stops when the kernel restarts")


✅ Server running on port 8765
🔗 App URL: https://8765-gpu-t4-s-z6oy5o8lveqc-c.asia-southeast1-0.prod.colab.dev
📌 Keep this cell running — the server stops when the kernel restarts
